<a href="https://colab.research.google.com/github/abdullahzunorain/About--RAG-Simple-RAG-App-Test/blob/main/Movie_Analysis_RAG_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**API Key**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
API_KEY = "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"

**Installation**

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 54.7 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers


Using  **LLAMA Model**  and  **API Key**  for Chat with Application

In [ ]:
import os

from groq import Groq

client = Groq(
    api_key= API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Fast language models, also known as rapid language models or speed-focused language models, have gained significant attention in recent years due to their growing importance in various industries and applications. Here are some reasons why fast language models are crucial:

1. **Speed and Efficiency**: Fast language models are designed to process and generate text at a much faster rate than traditional language models. This is particularly important in applications where speed and efficiency are critical, such as chatbots, customer service platforms, and natural language processing (NLP) systems.
2. **Volume and Scale**: With the proliferation of user-generated content and social media, the amount of text data available has grown exponentially. Fast language models can efficiently process and analyze large volumes of text data, enabling organizations to extract insights, sentiments, and trends from vast amounts of data.
3. **Real-time Processing**: In many applications, such as real-ti

Reading **Trained Dataset** by using Pandas

**Movies Dataset:** https://docs.google.com/spreadsheets/d/1RXRuXlni1vzXgm0Egl6MysTHMM2zJR7bl8TKTRGnxfQ/edit?usp=sharing

In [ ]:
import pandas as pd

# Load your dataset into a DataFrame
pd = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Hydra-Movie-Scrape.csv")

# Use the head() method on the DataFrame to display the first 6 rows
df = pd.head(6)

# Print the DataFrame
df.head(3)

,Title,Year,Summary,Short Summary,Genres,IMDB ID,Runtime,YouTube Trailer,Rating,Movie Poster,Director,Writers,Cast
0,Patton Oswalt: Annihilation,2017,"Patton Oswald, despite a personal tragedy, pro...","Patton Oswalt, despite a personal tragedy, pro...",Uncategorized,tt7026230,66,4hZi5QaMBFc,7.4,https://hydramovies.com/wp-content/uploads/201...,Bobcat Goldthwait,Patton Oswalt,Patton Oswalt
1,New York Doll,2005,A recovering alcoholic and recently converted ...,A recovering alcoholic and recently converted ...,Documentary|Music,tt0436629,75,jwD04NsnLLg,7.9,https://hydramovies.com/wp-content/uploads/201...,Greg Whiteley,Arthur Kane,Sylvain Sylvain
2,Mickey's Magical Christmas: Snowed in at the H...,2001,After everyone is snowed in at the House of Mo...,Mickey and all his friends hold their own Chri...,Adventure|Animation|Comedy|Family|Fantasy,tt0300195,65,uCKwHHftrU4,6.8,https://hydramovies.com/wp-content/uploads/201...,Tony Craig,Thomas Hart,Carlos Alazraqui|Wayne Allwine


**Application Generation**

In [ ]:
import pandas as pd
import numpy as np
from groq import Groq
import faiss
from sentence_transformers import SentenceTransformer

# Step 1: Initialize the Groq API client with your API key

client = Groq(api_key=API_KEY)

# Step 2: Load your dataset
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Hydra-Movie-Scrape.csv")

# Step 3: Handle missing values in the "Summary" column by replacing NaN with empty strings
data['Summary'].fillna('', inplace=True)

# Step 4: Initialize the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Step 5: Embed the "Summary" column
embeddings = embedder.encode(data['Summary'].tolist(), show_progress_bar=True)

# Step 6: Store the embeddings as a list of arrays in the DataFrame
data['summary_embeddings'] = list(embeddings)

# Step 7: Initialize FAISS index for vector search
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))  # Add embeddings to FAISS

# Step 8: Define a function to search for similar summaries using FAISS
def search_similar_summaries(query, top_n=5):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, top_n)
    return data.iloc[indices[0]]

# Step 9: Define a function to generate the augmented response using the Groq API
def generate_response(user_query):
    # Retrieve similar movie summaries from the dataset
    retrieved_data = search_similar_summaries(user_query, top_n=5)

    # Create context from the retrieved data to pass as input to the Groq API
    context = ""
    for idx, row in retrieved_data.iterrows():
        context += f"Title: {row['Title']}\nSummary: {row['Summary']}\nGenres: {row['Genres']}\nDirector: {row['Director']}\n\n"

    # Construct the final prompt for Groq's language model
    prompt = f"The user asked: {user_query}\nHere are some relevant movies:\n{context}"

    # Use the Groq API to get a completion from the language model
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # Access the generated response using dot notation
    generated_text = chat_completion.choices[0].message.content

    return generated_text


# Step 10: Example usage of the RAG application
user_query = "Man of steel"
response = generate_response(user_query)

# Output the generated response
print(response)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<ipython-input-13-e921cbecb6a3>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Summary'].fillna('', inplace=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/124 [00:00<?, ?it/s]

I see you're looking for movies with "Man of Steel" in the title or featuring the character. Let me highlight a few options from the list:

1. **Batman: The Dark Knight Returns, Part 2** - This animated movie features the Man of Steel (Superman) as a character, battling Batman and the Joker.
2. **Iron Man** - While not directly featuring the Man of Steel, this movie is part of the Marvel Cinematic Universe and explores the character's origins and technology.

The other movies on the list don't seem to feature the Man of Steel directly, but rather focus on other characters or themes. If you're specifically looking for Superman or DC Comics-related movies, I can try to provide more recommendations!
